# Recommendation System

## Business Understanding

For our final project, we have decided to create a custom recommendation system. The data chosen for training/testing came from the [Million Song Dataset Challenge](https://www.kaggle.com/c/msdchallenge#description). The dataset was originally provided by The Echo Nest and made available by [Columbia University](https://labrosa.ee.columbia.edu/millionsong/) with the puprose encouraging research in machine learning. With this dataset, we will attempt to predict which song a user will want to listen to next.

## Data Understanding

Importing the datasets:

In [1]:
import graphlab as gl
usage_data = gl.SFrame.read_csv("../data/10000.txt",
                                header=False,
                                delimiter='\t',
                                column_type_hints={'X3':int})

songs = gl.SFrame.read_csv("../data/song_data.csv")

This non-commercial license of GraphLab Create for academic use is assigned to kjprice@smu.edu and will expire on August 04, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1502938977.log


Unable to parse line "be941a98402c5f6f300fe4b48cc92b1f7e1bdb1c	SONEJIJ1"

1 lines failed to parse correctly

Finished parsing file /Users/kjprice/Projects/smu/smu-data-mining/DataMiningProject/data/10000.txt

...

Parsing completed. Parsed 100 lines in 0.85936 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 637410 lines. Lines per second: 414672

Finished parsing file /Users/kjprice/Projects/smu/smu-data-mining/DataMiningProject/data/song_data.csv

Parsing completed. Parsed 1000000 lines in 2.09136 secs.

In [2]:
%%capture

# Change data labels to be human readable
print(usage_data.rename({'X1':'user', 'X2': 'song', 'X3': 'plays'}))

Two datasets are included. The first one (`10000.txt`) has three fields: `user`, `song`, `plays` and describes how many times (`play`) a `song` is listened to by a `user`.

In [3]:
usage_data.head()

user,song,plays
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1


There is a second dataset which provides additional information for each songs

In [4]:
songs.head()

song_id,title,release,artist_name,year
SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze PresentsWinter Sessions ...,Der Mystic,0
SOZVAPQ12A8C13B63C,"Symphony No. 1 G minor""Sinfonie ...",Berwald: Symphonies Nos.1/2/3/4 ...,David Montgomery,0
SOQVRHI12A6D4FB2D7,We Have Got Love,Strictly The Best Vol. 34,Sasha / Turbulence,0
SOEYRFT12AB018936C,2 Da Beat Ch'yall,Da Bomb,Kris Kross,1993
SOPMIYT12A6D4F851E,Goodbye,Danny Boy,Joseph Locke,0
SOJCFMH12A8C13B0C2,Mama_ mama can't you see? ...,March to cadence with theUS marines ...,The Sun Harbor's Chorus-Documentary Recordings ...,0


## Train and Adjust parameters

## Evaluate and Compare

## Visualize Results

## Summarize the Ramifications

## Deployment

## Exceptional Work